## Combine GOES netCDFs into one .zarr file

This could also be done using the `goes-ortho.build_zarr()` function but that is currently not working.

Clinton Alden
27 March 2025

In [1]:
import xarray as xr
import os
import dask
from dask.distributed import Client

In [2]:
def goes_nc_to_zarr(in_dir, out_dir, out_name):    
# Directory containing the NetCDF files
    nc_dir = in_dir
    
    # Recursively list all NetCDF files in the directory and subdirectories
    nc_files = []
    for root, dirs, files in os.walk(nc_dir):
        for file in files:
            # print(file)
            if file.endswith('.nc'):
                nc_files.append(os.path.join(root, file))

    # Open multiple NetCDF files as a list of datasets
    datasets = [xr.open_dataset(f) for f in nc_files]

    # Concatenate datasets along the 't' coordinate
    combined_ds = xr.concat(datasets, dim='t')

    # Save the combined dataset to a Zarr file
    out_name = out_name
    combined_ds.to_zarr(out_dir+out_name)

In [3]:
for day in range(1,11):
    day_of_month = str(day)
    if day<10:
        out_day_of_month = '0' + day_of_month
    else:
        out_day_of_month = day_of_month
    print(f'Starting Feb {day_of_month}')
    in_dir = f'/storage/cdalden/goes/goes16/2023/2/{day_of_month}/'
    out_dir = '/storage/cdalden/goes/goes16/2023/'
    out_name = f'goes16_C02_colorado_202302{out_day_of_month}.zarr'

    goes_nc_to_zarr(in_dir, out_dir, out_name)
    print(f'Finished Feb {day_of_month}')

Starting Feb 1


/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Finished Feb 1
Starting Feb 2


/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Finished Feb 2
Starting Feb 3


...

Finished Feb 8
Starting Feb 9


/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Finished Feb 9
Starting Feb 10


/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Finished Feb 10


## Combine all zarr files for each date into one file

In [4]:
def combine_zarr_files(input_dir, output_file):
    # List all Zarr files in the input directory
    zarr_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.zarr')]

    # Open each Zarr file as an xarray.Dataset
    datasets = [xr.open_zarr(zarr_file) for zarr_file in zarr_files]

    # Concatenate datasets along the 't' dimension
    combined_ds = xr.concat(datasets, dim='t')

    # Save the combined dataset to a new Zarr file
    combined_ds.to_zarr(output_file, mode='w')

# Example usage
input_dir = '/storage/cdalden/goes/goes16/2023/'
output_file = '/storage/cdalden/goes/goes16/2023/goes16_C02_colorado_202306_combined.zarr'

combine_zarr_files(input_dir, output_file)

ValueError: Specified zarr chunks encoding['chunks']=(288,) for variable named 'max_radiance_value_of_valid_pixels' would overlap multiple dask chunks ((288, 288, 288, 288, 288, 288, 288, 288, 287, 288, 288, 288, 288, 288, 288, 288, 288, 288, 286),) on the region (slice(None, None, None),). Writing this array in parallel with dask could lead to corrupted data. Consider either rechunking using `chunk()`, deleting or modifying `encoding['chunks']`, or specify `safe_chunks=False`.